## Preambule

In [23]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

In [24]:
from Functions import (gdp_future, pop_func, emis_func, determine_coefficient,
                       popshare_func, emisshare_func, emis_total_func, rho, cumpopshare_func,
                       cumfuturepop_func, emis_f_func)

## Parameter

In [25]:
hist_startyear = 1990

## Paths

In [26]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF/WP5/")
path_data = path_main / "Data" / "EffortSharing" / str(hist_startyear)
path_figs = path_main / "Figures"

## Read data files

In [27]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [28]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [29]:
df_countries = pd.read_csv("X:/user/dekkerm/Data/AllCountries.csv")

## Preprocess

In [30]:
cs = np.zeros(shape=(6, len(all_countries_iso)))
for r_i, r in enumerate(all_countries_iso):
    for es_i, es in enumerate(["GF", "PC", "PCC", "A1_gdp_net", "A1_hdi_net", "A2_gdp_net"]):
        try:
            perc = 0#0.20*float(xr_total.sel(ISO=r, Time=2019).GHG_p)
            year = int(xr_budgets_scenario.Time[np.where(xr_budgets_scenario.sel(Region=r, Variable=es, Ccat="C1+C2").Value <= perc)[0]][0])
        except:
            year = 2101
        cs[es_i, r_i] = year
dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs[0]
dicty["C_PC"] = cs[1]
dicty["C_PCC"] = cs[2]
dicty["C_a1g_net"] = cs[3]
dicty["C_a1h_net"] = cs[4]
dicty["C_a2g_net"] = cs[5]
df_nz = pd.DataFrame(dicty)
median = np.median(df_nz['C_GF']) # Correct for missing data or negative current emissions
df_nz["C_GF"] = median

## Plot

In [31]:
fig = go.Figure()

ts = ['<1.5 (50%)', '1.5 (50%, return)', '<2 (67%)', '<2 (50%)', '<2.5 (50%)', '<3 (50%)', '<4 (50%)', '>4 (50%)']

fig.add_trace(
    go.Choropleth(
    locations=df_nz['Region'],
    z = df_nz["C_GF"],
    zmin = 2019,
    zmax = 2100,
    locationmode = 'ISO-3',
    colorscale = 'Spectral_r', 
    text = df_nz["C_GF"],
    hovertemplate  = '%{text}',
    name="",
    colorbar=dict(len=1, x=1, title={"text":"Year"})
))
fig.update_geos(showocean=True, oceancolor="whitesmoke")
fig.update_geos(showlakes=True, lakecolor="whitesmoke")

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.6,
            y=-0.05,
            showactive=True,
            buttons=list(
                [
                    dict(
                        label="Grandfathering",
                        method="update",
                        args=[{"z": [df_nz['C_GF'].astype(float)],
                               "text": [df_nz['C_GF'].astype(float)],
                               }],
                    ),
                    dict(
                        label="Per-capita",
                        method="update",
                        args=[{"z": [df_nz['C_PC'].astype(float)],
                               "text": [df_nz['C_PC'].astype(float)],
                               }],
                    ),
                    # dict(
                    #     label="Per-capita Convergence",
                    #     method="update",
                    #     args=[{"z": [df_nz['C_PCC'].astype(float)],
                    #            "text": [df_nz['C_PCC'].astype(float)],
                    #            }],
                    # ),
                    dict(
                        label="Approach 1 - GDP",
                        method="update",
                        args=[{"z": [df_nz['C_a1g_net'].astype(float)],
                               "text": [df_nz['C_a1g_net'].astype(float)],
                               }],
                    ),
                    dict(
                        label="Approach 1 - HDI",
                        method="update",
                        args=[{"z": [df_nz['C_a1h_net'].astype(float)],
                               "text": [df_nz['C_a1h_net'].astype(float)],
                               }],
                    ),
                    dict(
                        label="Approach 2 - GDP",
                        method="update",
                        args=[{"z": [df_nz['C_a2g_net'].astype(float)],
                               "text": [df_nz['C_a2g_net'].astype(float)],
                               }],
                    ),
                ]
            ),
        )
    ]
)
fig.update_layout(height=800, width=1500)
#fig.update_layout(title_text = "Year in which 80% of GHG emissions (in 2019) are reduced (in C1 scenarios)")
fig.show()
fig_year = fig

In [32]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/Figure_Netzero.svg")
#fig.write_image("X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/Figure_Netzero.png", scale=3)

import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/HTMLs/Figure_Netzero.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF/WP5/Figures/Paper_effortsharing/HTMLs/Figure_Netzero.html', 'a') as f:
    #f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))